In [1]:
from matplotlib import pyplot as plt
import seaborn as sns

import pandas as pd
import datetime
import swifter

from tqdm.notebook import tqdm

In [2]:
from functools import reduce

In [3]:
import numpy as np

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model, np_utils

In [6]:
import tensorflow as tf

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
path_to_data = '../data/'

In [9]:
filename = 'part-00000'

In [10]:
data_df = pd.read_csv(path_to_data + 'sessions_features/' + filename + '.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/sessions_features/part-00000.csv'

In [ ]:
data_df.info()

In [ ]:
last_application_id_le = LabelEncoder()
last_application_id_le.fit(data_df['last_application_id'])
data_df['last_application_id'] = last_application_id_le.transform(data_df['last_application_id'])

In [ ]:
last_event_category_le = LabelEncoder()
last_event_category_le.fit(data_df['last_event_category'])
data_df['last_event_category'] = last_event_category_le.fit_transform(data_df['last_event_category'])

In [ ]:
target_le = LabelEncoder()
target_le.fit(data_df['multi_class_target'].dropna().values)

In [ ]:
train_df = data_df[pd.notnull(data_df['multi_class_target'])]
to_predict = data_df[~pd.notnull(data_df['multi_class_target'])]

In [ ]:
train_df.info()

In [ ]:
train_df['multi_class_target'] = target_le.transform(train_df['multi_class_target'])

In [ ]:
train_df.head(1)

In [ ]:
X_df = train_df.drop(['client', 'session_id', 'max_timestamp', 'multi_class_target'], axis=1)
y_df = train_df[['multi_class_target']]

In [ ]:
dummy_y = np_utils.to_categorical(y_df)

In [ ]:
X_df.shape

In [ ]:
dummy_y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, dummy_y, test_size=0.2, random_state=42)

In [ ]:
epochs = 20
batch = 512

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_df.shape[1],)))
model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.3))
model.add(Dense(256, activation="relu"))
# model.add(Dropout(0.3))
model.add(Dense(1024, activation="relu"))
# model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
# model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dense(dummy_y.shape[1], activation='softmax'))
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(lr=0.001),
    metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(X_test,y_test)
)

predict part

In [ ]:
y_pred = model.predict(to_predict.drop(['client', 'session_id', 'max_timestamp', 'multi_class_target'], axis=1), batch_size=batch)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
predicted_labels = target_le.inverse_transform(y_pred)

In [ ]:
to_predict['multi_class_target'] = predicted_labels

In [ ]:
to_predict.info()

In [ ]:
train_df['multi_class_target'] = target_le.inverse_transform(train_df['multi_class_target'])

In [ ]:
result_df = pd.concat([train_df, to_predict], axis=0, ignore_index=True)

In [ ]:
result_df.head(1)

In [ ]:
result_df.to_csv(path_to_data + 'predicted_sessions_results/' + filename + '.csv', index=False)